In [1]:
import os

os.environ["KERAS_BACKEND"] = "jax"

In [2]:
import keras
import keras_nlp

keras.config.set_floatx("bfloat16")
pali_gemma_lm = keras_nlp.models.PaliGemmaCausalLM.from_preset("pali_gemma_3b_224")

2024-05-27 02:16:01.132037: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-27 02:16:16.759562: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-05-27 02:16:16.761892: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [3]:
pali_gemma_lm.summary()

Preprocessor: "pali_gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ pali_gemma_tokenizer (PaliGemmaTokenizer)          │                                             257,152 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "pali_gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ images (InputLayer)           │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ response_mask (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pali_gemma_backbone           │ (None, None, 2048)        │   2,923,335,408 │ images[0][0],              │
│ (PaliGemmaBackbone)           │                           │                 │ padding_mask[0][0],        │
│                               │                           │                 │ response_mask[0][0],       │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 257152)      │     526,647,296 │ pali_gemma_backbone[0][0]  │
│ (ReversibleEmbedding)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, None, 257152)      │               0 │ token_embedding[1][0]      │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,923,335,408 (5.45 GB)

 Trainable params: 2,923,335,408 (5.45 GB)

 Non-trainable params: 0 (0.00 B)

In [4]:
import PIL.Image
import io
import requests
import numpy as np


def read_image(url):
    contents = io.BytesIO(requests.get(url).content)
    image = PIL.Image.open(contents)
    image = np.array(image)
    # Remove alpha channel if neccessary.
    if image.shape[2] == 4:
        image = image[:, :, :3]
    return image

In [5]:
image_url = "https://storage.googleapis.com/keras-cv/models/paligemma/cow_beach_1.png"
image = read_image(image_url)

In [8]:
prompt = "answer en where is the cow standing?\n"
output = pali_gemma_lm.generate(
    inputs={
        "images": image,
        "prompts": prompt,
    }
)
print(output)

answer en where is the cow standing?
beach


In [10]:
prompts = [
    "answer en where is the cow standing?\n",
    "answer en what color is the cow?\n",
    "describe en\n",
    "detect cow\n",
    "segment cow\n",
]
images = [image, image, image, image, image]
outputs = pali_gemma_lm.generate(
    inputs={
        "images": images,
        "prompts": prompts,
    }
)
for output in outputs:
    print(output)

answer en where is the cow standing?
beach
answer en what color is the cow?
red
describe en
A cow on the beach
detect cow
<loc0274><loc0000><loc1022><loc0691> cow
segment cow
<loc0279><loc0000><loc0957><loc0691> <seg014><seg039><seg092><seg116><seg007><seg055><seg089><seg062><seg005><seg121><seg060><seg121><seg016><seg118><seg019><seg068> cow


In [11]:
import numpy as np

image = np.random.uniform(-1, 1, size=(224, 224, 3))
x = {
    "images": [image, image],
    "prompts": ["answer en Where is the cow standing?\n", "caption en\n"],
}
y = {
    "responses": ["beach", "A brown cow standing on a beach next to the ocean."],
}

pali_gemma_lm = keras_nlp.models.PaliGemmaCausalLM.from_preset("pali_gemma_3b_224")
pali_gemma_lm.fit(x=x, y=y, batch_size=2)

2024-05-27 02:31:02.458598: W external/tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1004.50MiB (rounded to 1053294592)requested by op 
2024-05-27 02:31:02.459507: W external/tsl/tsl/framework/bfc_allocator.cc:494] ***********************************************************************************************_____
E0527 02:31:02.459562    4868 pjrt_stream_executor_client.cc:2826] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 1053294592 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation: 1004.50MiB
              constant allocation:         0B
        maybe_live_out allocation: 1004.50MiB
     preallocated temp allocation:         0B
                 total allocation:    1.96GiB
              total fragmentation:         0B (0.00%)
Peak buffers:
	Buffer 1:
		Size: 1004.50MiB
		Entry Parameter Subshape: bf16[257152,2048]

	Buffer 2:
		Size: 1004.5

ValueError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 1053294592 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation: 1004.50MiB
              constant allocation:         0B
        maybe_live_out allocation: 1004.50MiB
     preallocated temp allocation:         0B
                 total allocation:    1.96GiB
              total fragmentation:         0B (0.00%)
Peak buffers:
	Buffer 1:
		Size: 1004.50MiB
		Entry Parameter Subshape: bf16[257152,2048]
		==========================

	Buffer 2:
		Size: 1004.50MiB
		XLA Label: fusion
		Shape: bf16[257152,2048]
		==========================

	Buffer 3:
		Size: 4B
		Entry Parameter Subshape: f32[]
		==========================

